# Vantage6

In [ ]:
import time
from vantage6.client import Client
from sklearn.linear_model import LogisticRegression
from src import config

## Installing a data node

In this tutorial, we do not intend to create data nodes as an exercise, since the content is focused on developing and using federated algorithms. In any case, you will see a live demo of how a data node can be created, which will follow the steps below.

You can set up a vantage6 data node with a few commands. The only requirements are `python3` and `docker`. You should do the following steps to create a node:

1. Make a virtual environment
```shell
    python -m venv .venv
```
2. Install vantage6 in this environment
```shell
    source .venv/bin/activate
    pip install vantage6==3.11.1
```
3. Create the node
```shell
    vnode new
```

In the last step, you should follow the prompt and provide the input you are asked for. Many of the inputs are information you receive from the central node administrator.

After creating a node, you can bring it online as follows:

4. Start the node
```shell
    vnode start --attach
```

The `attach` tag can be used to follow the logs, which can be very useful.

If you do not expect that the node will receive tasks, it is common practice to bring it down, which you can do as follows:

5. Stopping the node
```shell
    vnode stop
```

Finally, you can check other node commands:

6. Getting help
```shell
    vnode help
```

## Sending a task

Now that we have _real_ data nodes, you can send tasks to them. We will send a logistic regression task that does exactlly the same as we did in the previous tutorial. The difference is that all the steps are automated, which means that we can train the logistic regression until model convergence is achieved or at least for many more steps than we did manually.

Before starting, you need to edit the `config.py` file, which is located in the `src` directory. The information you need to provide in this file will be shared with you during the lecture.

In [ ]:
# Initialize the vantage6 client object, and run the authentication
client = Client(
    config.server_url, config.server_port, config.server_api,
    verbose=True
)
client.authenticate(config.username, config.password)
client.setup_encryption(None)

In [ ]:
# Vantage6 logistic regression task input
input_ = {
    'method': 'master',
    'master': True,
    'kwargs': {
        'org_ids': config.org_ids,
        'predictors': config.predictors,
        'outcome': config.outcome,
        'classes': config.classes,
        'max_iter': config.max_iter,
        'delta': config.delta,
    }
}

In [ ]:
# Vantage6 logistic regression task creation
task = client.task.create(
    collaboration=config.collaboration,
    organizations=config.org_ids,
    name='v6-logistic-regression-py',
    image=config.image,
    description='logistic regression',
    input=input_,
    data_format='json'
)

In [ ]:
# Retrieving results
task_info = client.task.get(task['id'], include_results=True)
while not task_info.get('complete'):
    print('Waiting for results...')
    task_info = client.task.get(task['id'], include_results=True)
    time.sleep(1)
result_info = client.result.list(task=task_info['id'])
results = result_info['data'][0]['result']

In [ ]:
# EXERCISE
# Explore the results

- How do the result compare with the one you did manually?
- What happens if you fix the maximum number of iterations to 5?

## References

- [Vantage6 documentation](https://docs.vantage6.ai/en/main/)